In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from preprocessing import preprocessing

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import ExtraTreeRegressor

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import BaggingRegressor

from xgboost import XGBRegressor

### Funciones Auxiliares

In [3]:
def leer_csv(ruta):
    return pd.read_csv(ruta)

In [4]:
def separar_train_test(df_train, logaritmico):
    X = df_train.copy().drop(columns = ['precio', 'log_precio'])
    if logaritmico:
        y = df_train['log_precio']
    else:
        y = df_train['precio']
    return train_test_split(X, y, test_size = TEST_SIZE, random_state = RANDOM_STATE)
    

## Función de entrenamiento para los modelos

In [10]:
def entrenar(modelo, ruta_df_train, nombre_modelo, logaritmico):

    df_train = leer_csv(ruta_df_train)
    
    X_train, X_test, Y_train, Y_test = separar_train_test(df_train, logaritmico)
    
    resultado = []
    

    print('------------------------------------------------')
    print('Comienza a entrenar: '+ nombre_modelo)
    print()

    modelo = modelo   
    modelo.fit(X_train, Y_train)
    prediccion = modelo.predict(X_test)

    if logaritmico:
        score = mean_absolute_error(np.exp(prediccion), np.exp(Y_test))
    else:
        score = mean_absolute_error(prediccion, Y_test)

    resultado.append((str(nombre_modelo), str(score)))

    print('**FINALIZADO** ' + nombre_modelo + ' Score [MAE]: ' + str(score))
    print()
    return resultado, modelo



In [11]:
TEST_SIZE = 0.1
RANDOM_STATE = 42

RUTA_TRAIN_CON_NULLS = 'data/train_preproc_con_nulls.csv'
RUTA_TRAIN_SIN_NULLS = 'data/train_preproc_sin_nulls.csv'
RUTA_TEST_CON_NULLS = 'data/test_preproc_con_nulls.csv'
RUTA_TEST_SIN_NULLS = 'data/test_preproc_sin_nulls.csv'

CON_PRECIO_LOGARITMICO = True

## XGBoost

In [15]:
modelo_xgboost = XGBRegressor(n_estimators=150,max_depth=21,subsample=0.9,min_child_weight =2,colsample_bytree = 0.8,eta = 0.07,n_jobs = 8,min_split_loss = 0)
nombre_xgboost = 'XGBoost'

In [16]:
resultado, modelo_xgboost =  entrenar(modelo_xgboost, RUTA_TRAIN_CON_NULLS, nombre_xgboost, CON_PRECIO_LOGARITMICO)

------------------------------------------------
Comienza a entrenar: XGBoost

[21:53:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
**FINALIZADO** XGBoost Score [MAE]: 485483.319739488



**FINALIZADO** XGBoost Score: 543519.3351068652

**FINALIZADO** XGBoost Score: 543808.3471162983 //Antes de agregar metros_cubiertos

**FINALIZADO** XGBoost Score: 500191.9174657849 //Despues de agregar metros_cubiertos

**FINALIZADO** XGBoost Score: 501766.49058353336 //Despues de agregar gimnasio, usos multiples y piscina

**FINALIZADO** XGBoost Score: 500862.1657935754 // Agrego solo piscina.

**FINALIZADO** XGBoost Score: 499181.4921723928 //Agrego precio promedio por m2 por provincia. Dejando piscina mejora.

**FINALIZADO** XGBoost Score: 496056.96402535716 //Tony agrega lat+lng precio promedio por m2

**FINALIZADO** XGBoost Score: 495398.1192696873 //Agrego feature tiene_sup_descubierta

**FINALIZADO** XGBoost Score: 491674.73406999116 //Agrego feature diff_metros_totales y metros_cubiertos

**FINALIZADO** XGBoost Score: 491335.880882687 //Agrego one hot encoding para antiguedad

**FINALIZADO** XGBoost Score: 490347.0285999908 //Dejando columna antiguedad Y el one hot encoding (antes se dropeaba)

**FINALIZADO** XGBoost Score: 489407.5245960651 //No imputando. Poniendo listas vacias

**FINALIZADO** XGBoost Score: 487797.2694374844 // **ENTREGA** USANDO 0.1 DE TEST_SIZE

**FINALIZADO** XGBoost Score:485399.4606591214 // con palabras [increíble excelente hermos espectacular exclusivo] para palabras importantes

**FINALIZADO** XGBoost Score: 482156.3098664535 //con palabras ['cuarto servicio','para auto','baño completo','sala comedor','cocina integral','baño vestidor'] para palabras importantes

**FINALIZADO** XGBoost Score: 480918.3753545355 //con palabras ['cuarto servicio','para auto','baño completo','baño vestidor','exclusivo'] para palabras importantes

**FINALIZADO** XGBoost Score: 475666.92711221 // **Entrege** con 0.05 DE TEST_SIZE con palabras ['roof garden', 'doble altura', 'sala tv', \
           'lugares estacionamiento','cajones estacionamiento',\
            'family room','double altura','salón juego','ideal para',\
            'amplios espacio', 'independiente', 'casa condominio',\
            'salón juego', 'pisos madera','exclusivo'] para palabras importantes

--------------------------------------------------------------------

**FINALIZADO** XGBoost Score [MAE]: 486560.74267065345 // One hot encoding de habitaciones y banos






## RandomForest Regressor

In [17]:
modelo_random_forest = RandomForestRegressor(n_estimators = 200, max_depth = 500, n_jobs = -1)
nombre_random_forest = 'RandomForestRegressor'

resultado, modelo_random_forest =  entrenar(modelo_random_forest, RUTA_TRAIN_SIN_NULLS, nombre_random_forest, CON_PRECIO_LOGARITMICO)

#500683.88162094826

------------------------------------------------
Comienza a entrenar: RandomForestRegressor

**FINALIZADO** RandomForestRegressor Score [MAE]: 500784.9079311183



**FINALIZADO** RandomForestRegressor Score [MAE]: 500686.94137633865 // One hot encoding banos y habitaciones

_______________________
## Función para entrega en kaggle

In [19]:
def predecir(modelo, ruta_train, ruta_test):
    
    df_train = leer_csv(ruta_train).drop(columns = ['precio', 'log_precio'])
    to_predict = leer_csv(ruta_test)
    
    columnas_train = df_train.columns
    to_predict = to_predict.reindex(columns = columnas_train)
    
    predicciones = modelo.predict(to_predict)
    
    return predicciones

In [ ]:
predicciones_modelo_xgboost = predecir(modelo_xgboost, RUTA_TRAIN_CON_NULLS, RUTA_TEST_CON_NULLS)
predicciones_modelo_random_forest = predecir(modelo_random_forest, RUTA_TRAIN_SIN_NULLS, RUTA_TEST_SIN_NULLS)

In [ ]:
def crear_df_predicciones(lista_predicciones, son_logaritmicas):
    
    df_test_original = leer_csv('data/test.csv')
    Id = df_test_original['id']
    
    df_resultado = pd.DataFrame()
    
    df_resultado['id'] = Id
    
    i = 0
    for prediccion in lista_predicciones:
        if son_logaritmicas:
            df_resultado['target_' + str(i)] = np.exp(predicciones)
        else:
            df_resultado['target_' + str(i)] = predicciones
    
    return df_resultado

,Id,target
0,4941,7.509248e+06
1,51775,9.340728e+05
2,115253,2.335410e+06
3,299321,1.534850e+06
4,173570,6.473417e+05
5,30862,1.123369e+06
6,244471,2.395494e+06
7,127794,5.637885e+05
8,71558,2.240932e+06
9,218011,1.327764e+06
